In [1]:
from sklearn.metrics import classification_report
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn import preprocessing
from dask import dataframe as dd
from dask_ml import model_selection as md
from dask_ml import preprocessing as pr
import ipaddress as ip
from time import sleep

2022-12-23 11:14:47.296652: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-23 11:14:47.792345: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-23 11:14:47.863595: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-23 11:14:47.863615: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
months = {
  'Jan' : 1,
  'Feb' : 2,
  'Mar' : 3,
  'Apr' : 4,
  'May' : 5,
  'Jun' : 6,
  'Jul' : 7,
  'Aug' : 8,
  'Sep' : 9,
  'Oct' : 10,
  'Nov' : 11,
  'Dec' : 12
}

In [3]:
def convert_timestamp_c_to_python(timestamp: str) -> str:
  times = timestamp.split(' ')
  data = f'{times[2]}/{months[times[1]]}/{times[4]} {times[3]}'
  return data

In [4]:
def splitDate(row):
  time = row
  time_values = time.split(' ')
  time_date = time_values[0].split('/')
  time_complete = time_values[1].split(':')
  # time_half = time_values[2]
  time_bind = ''
  for date in time_date:
    time_bind += date
  for complete in time_complete:
    time_bind += complete
  # if time_half == "AM":
  #   time_bind += '0'
  # else:
  #   time_bind += '12'
  # print(time_bind)
  return int(time_bind)

In [5]:
string = 'Fri Dec 23 05:02:50 2022'
string = convert_timestamp_c_to_python(string)
string = splitDate(string)
print(string)

23122022050250


In [6]:
def ip_str_to_int(entrada):
  retorno = int(ip.ip_address(entrada))
  return retorno

In [7]:
model = tf.keras.models.load_model('trabalho_22_12_5f.h5')

2022-12-23 11:14:50.969351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-23 11:14:50.970221: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-23 11:14:50.970256: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (arch): /proc/driver/nvidia/version does not exist
2022-12-23 11:14:50.970820: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
csv = pd.read_csv('pcap.csv')

In [9]:
csv.head()

,Timestamp,Src IP,Dst IP,Src Port,Tot Fwd Pkts
0,Fri Dec 23 11:14:45 2022,52.114.195.26,192.168.5.144,165.0,178.0
1,Fri Dec 23 11:14:45 2022,0.0.0.0,255.255.255.255,148.0,164.0
2,Fri Dec 23 11:14:45 2022,52.114.195.26,192.168.5.144,165.0,172.0
3,Fri Dec 23 11:14:45 2022,192.168.5.144,52.114.195.26,282.0,90.0
4,Fri Dec 23 11:14:45 2022,52.114.195.26,192.168.5.144,165.0,181.0


In [10]:
string = csv["Timestamp"][0]
string = convert_timestamp_c_to_python(string)
string = splitDate(string)
print(string)

23122022111445


In [11]:
mip = csv["Src IP"][0]
mip = ip_str_to_int(mip)
print(mip)

879936282


In [12]:
csv.tail()

,Timestamp,Src IP,Dst IP,Src Port,Tot Fwd Pkts
1072,Fri Dec 23 11:14:50 2022,52.114.195.26,192.168.5.144,165.0,167.0
1073,Fri Dec 23 11:14:50 2022,192.168.5.144,52.114.195.26,282.0,94.0
1074,Fri Dec 23 11:14:50 2022,52.114.195.26,192.168.5.144,165.0,178.0
1075,Fri Dec 23 11:14:50 2022,52.114.195.26,192.168.5.144,165.0,181.0
1076,Fri Dec 23 11:14:50 2022,52.114.195.26,192.168,NaN,NaN


In [13]:
csv.drop(csv.tail(1).index,inplace=True)
csv.tail(1)

,Timestamp,Src IP,Dst IP,Src Port,Tot Fwd Pkts
1075,Fri Dec 23 11:14:50 2022,52.114.195.26,192.168.5.144,165.0,181.0


In [14]:
csv['Src Port'] = csv['Src Port'].astype(int)
csv['Tot Fwd Pkts'] = csv['Tot Fwd Pkts'].astype(int)

In [15]:
csv['Timestamp'] = csv['Timestamp'].apply(lambda value : splitDate(convert_timestamp_c_to_python(value)))

In [16]:
csv.tail()

,Timestamp,Src IP,Dst IP,Src Port,Tot Fwd Pkts
1071,23122022111450,52.114.195.26,192.168.5.144,165,150
1072,23122022111450,52.114.195.26,192.168.5.144,165,167
1073,23122022111450,192.168.5.144,52.114.195.26,282,94
1074,23122022111450,52.114.195.26,192.168.5.144,165,178
1075,23122022111450,52.114.195.26,192.168.5.144,165,181


In [17]:
csv['Src IP'] = csv['Src IP'].apply(lambda value : ip_str_to_int(value))
csv['Dst IP'] = csv['Dst IP'].apply(lambda value : ip_str_to_int(value))

In [18]:
csv.tail()

,Timestamp,Src IP,Dst IP,Src Port,Tot Fwd Pkts
1071,23122022111450,879936282,3232236944,165,150
1072,23122022111450,879936282,3232236944,165,167
1073,23122022111450,3232236944,879936282,282,94
1074,23122022111450,879936282,3232236944,165,178
1075,23122022111450,879936282,3232236944,165,181


In [19]:
print(model.predict(csv))

34/34 [==============================] - 0s 919us/step
[[0.011153]
 [0.011153]
 [0.011153]
 ...
 [0.011153]
 [0.011153]
 [0.011153]]


In [20]:
while True:
  sleep(1)
  teste = pd.read_csv('pcap.csv')
  teste.drop(teste.tail(1).index,inplace=True)
  teste['Src Port'] = teste['Src Port'].astype(int)
  teste['Tot Fwd Pkts'] = teste['Tot Fwd Pkts'].astype(int)
  teste['Src IP'] = teste['Src IP'].apply(lambda value : ip_str_to_int(value))
  teste['Dst IP'] = teste['Dst IP'].apply(lambda value : ip_str_to_int(value))
  teste['Timestamp'] = teste['Timestamp'].apply(lambda value : splitDate(convert_timestamp_c_to_python(value)))
  predict = model.predict(teste)
  print(predict, len(predict))

40/40 [==============================] - 0s 636us/step
[[0.011153]
 [0.011153]
 [0.011153]
 ...
 [0.011153]
 [0.011153]
 [0.011153]] 1279
45/45 [==============================] - 0s 617us/step
[[0.011153]
 [0.011153]
 [0.011153]
 ...
 [0.011153]
 [0.011153]
 [0.011153]] 1414
47/47 [==============================] - 0s 631us/step
[[0.011153]
 [0.011153]
 [0.011153]
 ...
 [0.011153]
 [0.011153]
 [0.011153]] 1481
51/51 [==============================] - 0s 625us/step
[[0.011153]
 [0.011153]
 [0.011153]
 ...
 [0.011153]
 [0.011153]
 [0.011153]] 1616
55/55 [==============================] - 0s 596us/step
[[0.011153]
 [0.011153]
 [0.011153]
 ...
 [0.011153]
 [0.011153]
 [0.011153]] 1750
59/59 [==============================] - 0s 610us/step
[[0.011153]
 [0.011153]
 [0.011153]
 ...
 [0.011153]
 [0.011153]
 [0.011153]] 1884
61/61 [==============================] - 0s 647us/step
[[0.011153]
 [0.011153]
 [0.011153]
 ...
 [0.011153]
 [0.011153]
 [0.011153]] 1952
64/64 [===========================

KeyboardInterrupt: 